In [174]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [176]:
pd.set_option('display.max_rows', 100, 'display.max_columns', None)

In [177]:
train_file = 'data/train_raw.csv'
# test_file = 'data/train_raw.csv'

In [178]:
import random
random.seed(42)

# read data
print('Reading CSV...')
t_df = pd.read_csv(train_file, index_col=0, dtype='float')
print('Done')

t_df['id'] = t_df['id'].astype(int)
t_df['id'] = t_df['id'].astype(int)

# split train-validation
sick = set(t_df[t_df['SepsisLabel'] == 1.0]['id'].unique())
healthy = set(t_df['id'].unique()) - sick
t_sick = set(random.sample(sick, int(len(sick) * 0.75)))
v_sick = sick - t_sick
t_healthy = set(random.sample(healthy, int(len(healthy) * 0.75)))
v_healthy = healthy - t_healthy

train = t_df[t_df.id.isin(list(t_sick) + list(t_healthy))].sort_values(['id', 'SepsisLabel'])
valid = t_df[t_df.id.isin(list(v_sick) + list(v_healthy))].sort_values(['id', 'SepsisLabel'])

Reading CSV...
Done


/tmp/ipykernel_7979/4184894311.py:12: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  t_sick = set(random.sample(sick, int(len(sick) * 0.75)))
/tmp/ipykernel_7979/4184894311.py:14: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  t_healthy = set(random.sample(healthy, int(len(healthy) * 0.75)))


# Under-sampling (maybe?)

In [179]:
remove_amount = len(t_healthy) - 2 * len(t_sick)
# remove_amount = 5000
remove_healthy = random.sample(healthy, remove_amount)
train = train[~train['id'].isin(remove_healthy)]

/tmp/ipykernel_7979/3267703958.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  remove_healthy = random.sample(healthy, remove_amount)


In [180]:
# train = train.drop(train.columns[8: 34], axis=1)
# valid = valid.drop(valid.columns[8: 34], axis=1)

In [181]:
train

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,id,Unit3,Gender_0.0,Gender_1.0
23.0,81.000000,100.000000,38.000000,136.500000,90.000000,71.000000,12.500000,35.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,65.79,0.0,0.0,-0.02,1.0,0.0,1,1.0,0.0,1.0
24.0,81.000000,100.000000,38.000000,136.500000,90.000000,71.000000,12.500000,35.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.500000,0.500000,0.000000,0.000000,0.0,0.000000,65.79,0.0,0.0,-0.02,2.0,0.0,1,1.0,0.0,1.0
25.0,81.750000,100.000000,38.000000,136.500000,90.000000,71.000000,12.125000,35.000000,0.333333,0.333333,0.333333,0.333333,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.333333,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.333333,0.000000,0.000000,0.0,0.000000,65.79,0.0,0.0,-0.02,3.0,0.0,1,1.0,0.0,1.0
26.0,81.230769,100.000000,38.082500,119.625000,87.750000,63.500000,12.038462,35.000000,0.250000,0.250000,0.500000,0.250000,0.250000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.250000,0.000000,0.000000,0.000000,0.0,0.0,0.250000,0.250000,0.000000,0.000000,0.0,0.000000,65.79,0.0,0.0,-0.02,4.0,0.0,1,1.0,0.0,1.0
27.0,81.750000,100.000000,38.082500,115.730769,81.692308,62.461538,12.012500,35.000000,0.200000,0.400000,0.400000,0.200000,0.200000,0.0,0.0,0.200000,0.0,0.200000,0.200000,0.200000,0.0,0.400000,0.200000,0.200000,0.200000,0.200000,0.0,0.0,0.400000,0.400000,0.200000,0.200000,0.0,0.200000,65.79,0.0,0.0,-0.02,5.0,0.0,1,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776714.0,92.990280,97.882567,36.447763,128.615857,89.877763,60.150472,15.603230,47.057456,0.078947,0.078947,0.236842,0.105263,0.078947,0.0,0.0,0.078947,0.0,0.052632,0.078947,0.078947,0.0,0.078947,0.000000,0.105263,0.026316,0.105263,0.0,0.0,0.078947,0.078947,0.052632,0.078947,0.0,0.078947,38.75,0.0,1.0,-0.05,38.0,0.0,19998,0.0,0.0,1.0
776715.0,73.451416,97.098386,37.161072,123.626040,73.992964,52.020070,15.955769,47.057456,0.076923,0.076923,0.230769,0.102564,0.076923,0.0,0.0,0.076923,0.0,0.051282,0.076923,0.076923,0.0,0.076923,0.000000,0.102564,0.025641,0.102564,0.0,0.0,0.076923,0.076923,0.051282,0.076923,0.0,0.076923,38.75,0.0,1.0,-0.05,39.0,0.0,19998,0.0,0.0,1.0
776716.0,73.451416,97.098386,37.161072,123.626040,73.992964,52.020070,15.955769,47.057456,0.075000,0.075000,0.225000,0.100000,0.075000,0.0,0.0,0.075000,0.0,0.050000,0.075000,0.075000,0.0,0.075000,0.000000,0.100000,0.025000,0.100000,0.0,0.0,0.075000,0.075000,0.050000,0.075000,0.0,0.075000,38.75,0.0,1.0,-0.05,40.0,0.0,19998,0.0,0.0,1.0
776717.0,73.050176,97.010936,37.161072,124.847282,76.665763,54.668775,15.995084,47.057456,0.073171,0.073171,0.219512,0.097561,0.073171,0.0,0.0,0.073171,0.0,0.048780,0.073171,0.073171,0.0,0.073171,0.000000,0.097561,0.024390,0.097561,0.0,0.0,0.073171,0.073171,0.048780,0.073171,0.0,0.073171,38.75,0.0,1.0,-0.05,41.0,0.0,19998,0.0,0.0,1.0


In [182]:
scaling_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2', 'Age', 'HospAdmTime', 'ICULOS']  # rest are already scaled
scaler = MinMaxScaler()
train[scaling_columns] = scaler.fit_transform(train[scaling_columns])
valid[scaling_columns] = scaler.transform(valid[scaling_columns])

In [183]:
train_y = train['SepsisLabel'].astype(int)
train_X = train.drop(['SepsisLabel'], axis=1)

valid_y = valid['SepsisLabel'].astype(int)
valid_X = valid.drop(['SepsisLabel'], axis=1)

In [184]:
from patient_LSTM import TrainDataset, TestDataset, patientLSTM, batch_collate, test_batch_collate

WINDOW = 30

train_ds = TrainDataset(train_X, train_y, WINDOW)
valid_ds = TrainDataset(valid_X, valid_y, WINDOW)


In [185]:
from torch.utils.data import DataLoader, WeightedRandomSampler

samples_weight = []
for patient in train_ds.patients:
    if patient in sick:
        samples_weight.append(1. / len(t_sick))
    else:
        samples_weight.append(1. / len(t_healthy))
samples_weight = torch.tensor(samples_weight)
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

# train_dl = DataLoader(train_ds, batch_size=8, collate_fn=batch_collate, sampler=sampler)
train_dl = DataLoader(train_ds, batch_size=8, collate_fn=batch_collate, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=8, collate_fn=batch_collate)

In [186]:
for i, (data, target) in enumerate(train_dl):
    print("batch index {}, class 0 {}, other classes {}".format(
        i,
        len(target[target == 0]),
        len(target[target == 1])))

batch index 0, class 0 8, other classes 0
batch index 1, class 0 6, other classes 2
batch index 2, class 0 6, other classes 2
batch index 3, class 0 6, other classes 2
batch index 4, class 0 7, other classes 1
batch index 5, class 0 7, other classes 1
batch index 6, class 0 6, other classes 2
batch index 7, class 0 7, other classes 1
batch index 8, class 0 6, other classes 2
batch index 9, class 0 7, other classes 1
batch index 10, class 0 8, other classes 0
batch index 11, class 0 6, other classes 2
batch index 12, class 0 7, other classes 1
batch index 13, class 0 7, other classes 1
batch index 14, class 0 6, other classes 2
batch index 15, class 0 8, other classes 0
batch index 16, class 0 5, other classes 3
batch index 17, class 0 8, other classes 0
batch index 18, class 0 8, other classes 0
batch index 19, class 0 6, other classes 2
batch index 20, class 0 6, other classes 2
batch index 21, class 0 8, other classes 0
batch index 22, class 0 6, other classes 2
batch index 23, class

In [187]:
from score_functions import F1
from LSTMTrainer import RNNTrainer
from sklearn.metrics import f1_score

model = patientLSTM(features_dim=42, hidden_dim=256, n_layers=2, dropout=0.3)

loss_fn = nn.BCEWithLogitsLoss()
score_fn = F1()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, betas=(0.9, 0.999))

epochs = 100

trainer = RNNTrainer(model, loss_fn, optimizer)

trainer.fit(train_dl, valid_dl, epochs, score_fn=score_fn, checkpoints=f'models/LSTM_window{WINDOW}', early_stopping=15)

--- EPOCH 1/100 ---


train_batch:   0%|          | 0/765 [00:00<?, ?it/s]

TP=tensor(252), FP=tensor(25), TN=tensor(5030), FN=tensor(809)


test_batch:   0%|          | 0/626 [00:00<?, ?it/s]

TP=tensor(86), FP=tensor(0), TN=tensor(4647), FN=tensor(268)

*** Saved checkpoint /home/student/Course-094295_hw1/models/LSTM_window30.pth
--- EPOCH 2/100 ---


...

train_batch:   0%|          | 0/765 [00:00<?, ?it/s]

TP=tensor(602), FP=tensor(60), TN=tensor(4995), FN=tensor(459)


test_batch:   0%|          | 0/626 [00:00<?, ?it/s]

TP=tensor(210), FP=tensor(67), TN=tensor(4580), FN=tensor(144)


FitResult(num_epochs=40, train_loss=[0.6748249530792236, 0.5492289066314697, 0.5694503784179688, 0.8599296808242798, 0.37858134508132935, 0.8712964057922363, 0.6704046726226807, 0.6745951175689697, 0.5763840079307556, 0.6128110289573669, 0.6029682755470276, 0.5271401405334473, 0.5029929876327515, 0.6667975187301636, 0.4713345468044281, 0.45606082677841187, 0.3090839982032776, 0.5311578512191772, 0.21663989126682281, 0.3795938193798065, 1.2013797760009766, 0.7355411648750305, 0.38243627548217773, 0.37462660670280457, 0.468042254447937, 0.38890108466148376, 0.3952289819717407, 0.4654213786125183, 0.3918718695640564, 0.5774655342102051, 0.3907599449157715, 0.4672580063343048, 0.38777869939804077, 0.1926313042640686, 0.583511233329773, 0.5889956951141357, 0.17239364981651306, 0.3801334500312805, 0.241241455078125, 0.30600857734680176, 0.8705578446388245, 0.6294603943824768, 0.6274059414863586, 0.14274325966835022, 0.29447925090789795, 0.6477545499801636, 1.0716660022735596, 0.6115663051605

In [188]:
torch.tensor([1]).squeeze(1).shape

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)